Парсинг статей с сайта медии.

Подбор медии для парсинга проводится вручную, так как каждая медия имеет свой сайт, чаще всего, с уникальным устройством (на многих сайтах нет архива статей).

Также код страницы каждой медии отличается и html-теги, которые нужно обработать необходимо указывать вручную, анализируя код страницы через инспектор кода в браузере.

P.S. для ускорения парсинга можно запускать до трёх скриптов в гугл колаб с одного аккаунта.



In [ ]:
# Вариант 1.
# Статьи в архиве на сайте медии представлены в виде
# Заголовок - Текст статьи

In [ ]:
!pip install translate

In [ ]:
!pip install googletrans==4.0.0-rc1 --upgrade

  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=29e2e46b1057d111f7370afc99ad092d243d62673f208f0c22ede393d5ecc5f9
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [1]:
from requests.exceptions import ChunkedEncodingError
import requests
from bs4 import BeautifulSoup
import json


# Функция для парсинга статей по ссылке

def art_parc(url_art, bias, counter, bias_num, fact, fact_num):
    url = url_art
    label = bias
    articles = []

    while True:
        try:
            headers = {
                "Accept-Language": "en-US,en;q=0.9,ja;q=0.8"
            }

            response = requests.get(url, headers=headers)
            response.raise_for_status()
            break
        except ChunkedEncodingError:
            print('ChunkedEncodingError occurred. Retrying...')

    soup = BeautifulSoup(response.content, 'html.parser')

    news_list = soup.find_all('div', class_='article-details')

   # translator = Translator(from_lang='en', to_lang='ja')

    for news in news_list:
        if news.find("div", class_="article-title") is None:
            print('Skipping this news item.')
            continue

        title = news.find("div", class_="article-title").get_text().strip()
      #  translated_title = translator.translate(title)

        if news.find("div", class_="article-body") is None:
            text = title
           # translated_text = translated_title
        else:
            text = news.find("div", class_="article-body").get_text().strip()
           # translated_text = translator.translate(text)

        article = {
            'id': counter,
            'title': title,
            'content': text,
            'label': [bias, fact],
            'label_num': [bias_num, fact_num]

        }
        counter += 1
        print(article)
        articles.append(article)

    return articles

In [4]:

# парсинг всех страницы из архива

for i in range(957, 1200):
  print(i)
  url = f"https://www.japantimes.co.jp/morearticles/world/?pgno={i}"
  data = art_parc(url, "center", 13 * i, 1, "high", 1)
 # print(data)
  with open(f"jap_center_h2.json", "a", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


Выходные данные были обрезаны до нескольких последних строк (5000).
{'id': 13108, 'title': 'Kishida unveils ¥17 trillion stimulus package amid pushback', 'content': 'Kishida unveils ¥17 trillion stimulus package amid pushback', 'label': ['center', 'high'], 'label_num': [1, 1]}
{'id': 13109, 'title': 'The risers — and fallers — in the latest sumo rankings', 'content': 'The risers — and fallers — in the latest sumo rankings', 'label': ['center', 'high'], 'label_num': [1, 1]}
{'id': 13110, 'title': 'Japan top envoy faces diplomatic test on trip to Israel and Jordan', 'content': 'Japan top envoy faces diplomatic test on trip to Israel and Jordan', 'label': ['center', 'high'], 'label_num': [1, 1]}
{'id': 13111, 'title': 'Hotly debated tax cuts risk backfiring on Kishida administration', 'content': 'Hotly debated tax cuts risk backfiring on Kishida administration', 'label': ['center', 'high'], 'label_num': [1, 1]}
{'id': 13112, 'title': "For Atsuki Yuasa, all it took was one pitch to spur Ti

In [ ]:
# Вариант 2.
# Статьи в архиве на сайте медии представлены в виде
# Только Заголовок статьи
# В этом случае нужно перейти по ссылке на текст статьи и спарсить его
# с вложенной соответственно страницы

# В остальном алгоритм аналогичен

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from requests.exceptions import ChunkedEncodingError

def art_parc_2(url_art, bias, counter, bias_num, fact, fact_num):
  url = url_art
  label = bias
  articles = []

  while True:
      try:
          response = requests.get(url)
          response.raise_for_status()
          break
      except ChunkedEncodingError:
          print('ChunkedEncodingError occurred. Retrying...')

  soup = BeautifulSoup(response.content, 'html.parser')
 # print(soup)
  news_left_list = soup.find_all('div', class_='text')
 # print(news_left_list)
  for news in news_left_list:
    # print(news)
      if news.find("a") is None:
          print('Skipping this news item.')
          continue

      link = news.find("a").get("href")
      title = news.get_text().strip()
      link = "https://www.publico.es" + link.replace("https://www.publico.es", '')
      #print(link)
      while True:
        try:
            response2 = requests.get(link)
            response2.raise_for_status()
            break
        except ChunkedEncodingError:
            print('ChunkedEncodingError occurred. Retrying...')

      soup2 = BeautifulSoup(response2.content, 'html.parser')

     # print(title)
      if soup2.find('div', class_='article-text') is None:
            text = title
      else:
          # print("GOOD CONT")
           text = soup2.find('div', class_="article-text").get_text().strip()


    #  print(text)

      article = {
              'id': counter,
              'title': title,
              'content': text,
              'bias': bias,
              'bias (numeric)': bias_num,
            # для датасета BF верхние две строчки раскомментировать
              'factuality': fact,
              'factuality (numeris)': fact_num,
      }
      print(article)
      counter += 1
      articles.append(article)
 # print(articles)
  return articles


In [ ]:

# парсинг всех страницы из архива

for i in range(1, 2):
  print(i)
  url = f"https://www.altnews.in/hindi/topics/politics/page/{i}/"
  data = art_parc(url, "left", 20 * i, 0, "high", 1)
 # print(data)
  with open(f"hindi_left_h1.json", "a", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


Пост обработка данных

In [ ]:
# Разделение единого json файла на отдельные (для каждой статьи свой)
# присваивание рандомного id в название и поле id json файла статьи

import json
import random
import string
import os

label = "dir"
if not os.path.exists(label):
    os.makedirs(label)
try:
    with open(f"{label}.json", "r", encoding="utf-8") as f:
        data = json.load(f)
except json.JSONDecodeError as e:
    error_position = e.pos  # Позиция символа, вызвавшего исключение
    with open(f"{label}.json", "r", encoding="utf-8") as f:
        json_data = f.read()
    start_index = max(0, error_position - 10)  # Начальный индекс для вывода
    error_context = json_data[start_index:error_position]  # Контекст ошибки
    print(f"Error occurred at position {error_position}:")
    print(error_context)

for item in data:
    item_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
    file_name = f"{item_id}.json"
    item["id"] = item_id
    with open(f"{label}/" + file_name, 'w') as file:
        json.dump(item, file, indent=4)

Скрипт для создания разных датасетов B, F из BF

In [ ]:
import os
import json

def process_files(directory):
    for filename in os.listdir(directory):
        print(filename)
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path) and filename.endswith('.json'):
            with open(file_path, 'r+') as file:
                data = json.load(file)
                if 'label' in data:

                    label = data['label']
                    if isinstance(label, list) and len(label) > 0:
                        data['label'] = label[1]  # Заменяем поле 'label' на первый элемент массива

                if 'label_num' in data:
                    label_num = data['label_num']
                    if isinstance(label_num, list) and len(label_num) > 0:
                        data['label_num'] = label_num[1]  # Заменяем поле 'label_num' на первый элемент массива

                # Обновляем файл с измененными данными
                file.truncate(0)
                file.seek(0)
                json.dump(data, file, indent=4)

# Указываем путь к директории
directory_path = 'dir'

process_files(directory_path)


Скрипт для сбора статистики

В данном случае проводится сбор статистики для комбинаций
bias-factuality в датасете BF

In [ ]:
import json
import os
import shutil

directory = 'BF'
# Получаем список файлов в текущей директории
files = os.listdir(directory)

# Создаем словарь для подсчета количества файлов по странам
parametr1_count = {}
parametr2_count = {}

# Проходимся по каждому файлу и считаем количество файлов по странам
for file in files:
    file = os.path.join(directory, file)
    if file.endswith('.json'):
        with open(file, 'r', encoding="utf-8") as f:
            data = json.load(f)

        parametr1 = tuple(data['label'])


        if parametr1 in parametr1_count:
            parametr1_count[parametr1] += 1
        else:
            print(parametr1)
            parametr1_count[parametr1] = 1

        # Копируем файл в директо рию stat_parametr1/{parametr1}, если страна соответствует заданной
        '''
        try:
            try:
                parametr1_dir = os.path.join(f'stat_f_{directory}', parametr1)
                if not os.path.exists(parametr1_dir):
                    os.makedirs(parametr1_dir)
                shutil.copy(file, parametr1_dir)
            except NotADirectoryError:
                break
        except OSError:
            break
        '''

# Выводим результаты подсчета
for parametr1, count in parametr1_count.items():
    print(f"{parametr1}: {count} файлов")



Соединение данных с allsides и с медий для датасета B

In [ ]:
# Переименовывание файлов в title, чтобы искать совпадения статей по заголовку и удалять повторы

import os
import json
import shutil

# Путь к директории с JSON-файлами
path = 'test_j2'

# Проходимся по всем файлам в директории
for filename in os.listdir(path):
    # Проверяем, что файл имеет расширение .json
    if filename.endswith(".json"):
        # Считываем содержимое файла
        with open(os.path.join(path, filename), "r", encoding='utf-8') as f:
            content = json.load(f)
        data = {
            "id": content['id'],
            "title": content["title"],
            "content": content["content"],
            "label": content["label"]
        }
        path2 = 'test_j2_news'
        try:
            name_file = content["id"]
          #  name_file = (content["title"]).replace("|", '').replace("'", "").replace("-", "").replace(".", "").replace(":", "").replace(" ", "_").replace("?", '').replace('/', '%').replace(",", "").replace("*", '').replace('\\', '').replace('"', '')
            with open(os.path.join(path2, name_file), "w", encoding='utf-8') as f:
                json.dump(data, f, indent=4)
        except OSError:
            print(name_file)
            print(f"{filename} with error")

In [ ]:
import os
import shutil

dir1 = 'test2'
dir2 = 'test_j2'
uniq_dir = 'uniq2'

# Создание директории "uniq2", если она не существует
if not os.path.exists(uniq_dir):
    os.makedirs(uniq_dir)

count = 0

for file1 in os.listdir(dir1):
    for file2 in os.listdir(dir2):
        if file1 == file2:
            print(file1)
            count += 1
            break
    else:
        src_path = os.path.join(dir1, file1)
        dst_path = os.path.join(uniq_dir, file1)
        shutil.move(src_path, dst_path)
        print(f"Файл {file1} перемещен в директорию {uniq_dir}")

print(count)